In [1]:
%cd /content/drive/MyDrive/Colab filtering/dataset

/content/drive/MyDrive/Colab filtering/dataset


In [2]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
rating=pd.read_csv("ratings.csv")
movies= pd.read_csv("movies.csv")
ratings=pd.merge(movies,rating).drop(['genres','timestamp'],axis=1)
print(ratings.head)

<bound method NDFrame.head of         movieId                                      title  userId  rating
0             1                           Toy Story (1995)       1     4.0
1             1                           Toy Story (1995)       5     4.0
2             1                           Toy Story (1995)       7     4.5
3             1                           Toy Story (1995)      15     2.5
4             1                           Toy Story (1995)      17     4.5
...         ...                                        ...     ...     ...
100831   193581  Black Butler: Book of the Atlantic (2017)     184     4.0
100832   193583               No Game No Life: Zero (2017)     184     3.5
100833   193585                               Flint (2017)     184     3.5
100834   193587        Bungo Stray Dogs: Dead Apple (2018)     184     3.5
100835   193609        Andrew Dice Clay: Dice Rules (1991)     331     4.0

[100836 rows x 4 columns]>


In [7]:
user_rating = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
print(user_rating.head())

title   '71 (2014)  ...  À nous la liberté (Freedom for Us) (1931)
userId              ...                                           
1              NaN  ...                                        NaN
2              NaN  ...                                        NaN
3              NaN  ...                                        NaN
4              NaN  ...                                        NaN
5              NaN  ...                                        NaN

[5 rows x 9719 columns]


In [11]:
#lest drop few movies which only few users have rated and fill other nan values with 0
user_rating= user_rating.dropna(thresh=10,axis=1).fillna(0)
print(user_rating)

title   'burbs, The (1989)  ...  ¡Three Amigos! (1986)
userId                      ...                       
1                      0.0  ...                    4.0
2                      0.0  ...                    0.0
3                      0.0  ...                    0.0
4                      0.0  ...                    0.0
5                      0.0  ...                    0.0
...                    ...  ...                    ...
606                    0.0  ...                    0.0
607                    0.0  ...                    0.0
608                    0.0  ...                    0.0
609                    0.0  ...                    0.0
610                    0.0  ...                    0.0

[610 rows x 2269 columns]


In [39]:
#buiild similarity matrix
item_similarity_df=user_rating.corr(method="pearson")
print(item_similarity_df.head())

title                              'burbs, The (1989)  ...  ¡Three Amigos! (1986)
title                                                  ...                       
'burbs, The (1989)                           1.000000  ...               0.353194
(500) Days of Summer (2009)                  0.063117  ...               0.125905
10 Cloverfield Lane (2016)                  -0.023768  ...               0.002733
10 Things I Hate About You (1999)            0.143482  ...               0.110612
10,000 BC (2008)                             0.011998  ...               0.083518

[5 rows x 2269 columns]


In [49]:
def get_similar_movies(movie_name,rating):
   similar_score = item_similarity_df[movie_name]*(rating-2.5)
   similar_score = similar_score.sort_values(ascending=False)
   
   return similar_score

In [54]:
#testing code
romantic_lover = [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similar_movies = pd.DataFrame()
for movie,rating in romantic_lover:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)

(500) Days of Summer (2009)                      2.584556
Alice in Wonderland (2010)                       1.395229
Silver Linings Playbook (2012)                   1.254800
Yes Man (2008)                                   1.116264
Adventureland (2009)                             1.112235
Marley & Me (2008)                               1.108381
About Time (2013)                                1.102192
Crazy, Stupid, Love. (2011)                      1.088757
50/50 (2011)                                     1.086517
Help, The (2011)                                 1.075963
Up in the Air (2009)                             1.053037
Holiday, The (2006)                              1.034470
Friends with Benefits (2011)                     1.030875
Notebook, The (2004)                             1.025880
Easy A (2010)                                    1.015771
Secret Life of Walter Mitty, The (2013)          0.997979
Perks of Being a Wallflower, The (2012)          0.967425
Toy Story 3 (2